# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**
#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [14]:
import pickle
import numpy as np
from scipy.sparse.linalg import svds
import json
import math
from scipy.sparse import csr_matrix
from collections import defaultdict
from utils import load_json, load_pkl
courses = load_json('data/courses_processed.txt')[0]           

3


In [15]:
# code from previous notebook computing TF-IDF matrix X 

# Extract the unique terms in course descriptions and names
unique_terms = list(set((' '.join([course['description'] for course in courses])).split(' ')))
course_names = [course['name'] for course in courses]

# Step 1: Tokenize and build vocabulary
def tokenize(text):
    return text.lower().split()
vocab = set()
doc_tokens = []
for course in courses:
    doc = course['description']
    tokens = tokenize(doc)
    doc_tokens.append(tokens)
    vocab.update(tokens)
vocab = sorted(vocab)  # consistent order
vocab_index = {term: i for i, term in enumerate(vocab)}

# Step 2: Compute term frequency (TF)
def compute_tf(tokens):
    tf = defaultdict(float)
    for word in tokens:
        tf[word] += 1
    total = len(tokens)
    for word in tf:
        tf[word] /= total
    return tf
tfs = [compute_tf(tokens) for tokens in doc_tokens]

# Step 3: Compute inverse document frequency (IDF)
def compute_idf(doc_tokens, vocab):
    N = len(course_names)
    idf = {}
    for word in vocab:
        df = sum(1 for tokens in doc_tokens if word in tokens)
        idf[word] = math.log((1 + N) / (1 + df)) + 1  # smoothed
    return idf
idf = compute_idf(doc_tokens, vocab)

# Step 4: Compute TF-IDF vectors for each document
def compute_tfidf(tf, idf, vocab_index):
    vec = np.zeros(len(vocab_index))
    for word, val in tf.items():
        if word in idf:
            vec[vocab_index[word]] = val * idf[word]
    return vec
doc_vectors = [compute_tfidf(tf, idf, vocab_index) for tf in tfs]

## Exercise 4.4: Latent semantic indexing

In [16]:
#Define data matrix and compute SVD 
matrix = np.vstack(doc_vectors)
matrix_sparse = csr_matrix(matrix.T, dtype=float)
U, S, Vt = svds(matrix_sparse, k=300)

In [17]:
#Verify correct dimentionality 
print(matrix_sparse.shape)
print(U.shape)
print(Vt.shape)

(10122, 854)
(10122, 300)
(300, 854)


- U is a (10122 X 300) term-topic matrix. Each column of U represents the contribution of all the 10122 terms on a main topic/concept.
- Vt is a (300 X 854) document-topic matrix. Each row of Vt represents the contribution of all the 854 document on a main topic/concept.  
- Finally, S is a (300 X 300) diagonal matrix containing on it's main diagonal the weights of the top 300 topics/concepts in increasing order.  

In [18]:
print("The top 20 singular values of X are: \n ", np.sort(S)[-20:][::-1])

The top 20 singular values of X are: 
  [3.93625354 3.52946105 2.92617437 2.42848641 1.63712864 1.60160462
 1.45895513 1.42724407 1.38418801 1.35372694 1.32973585 1.27306686
 1.24551032 1.2341388  1.19652518 1.16029898 1.14360026 1.14243644
 1.12273007 1.11779834]


## Exercise 4.5: Topic extarction


In [19]:
# reversing the matrices column or rows to have the eigenvectors in decreasing order
U = U[:,::-1]
S = S[::-1]
Vt = Vt[::-1, :]
S = np.diag(S)

# useful dictionaries for later functions
index_to_term = {i: term for term, i in vocab_index.items()}
name_index = {name: i for i, name in enumerate(course_names)}

In [20]:
# finds the most relevant topics based on a list of terms and course names
# params: numbrer_top_terms: number of most relevant terms to display
#         number_top_courses: number of most reéevant courses to display
#         number_topics: number of concepts/topic to calculate in such way
def find_topics(number_top_terms,number_top_courses, number_topics):

    for topic_idx in range(number_topics):
        print(f"\n🔹 Topic #{topic_idx + 1}")

        # Top terms for this topic
        top_term_indices = U[:,topic_idx].argsort()[::-1][:number_top_terms]
        top_terms = [index_to_term[i] for i in top_term_indices]
        print("  Top terms:", ", ".join(top_terms))

        # Top courses for this topic
        top_course_indices = Vt[topic_idx,:].argsort()[::-1][:number_top_courses]
        top_courses = [course_names[i] for i in top_course_indices]
        print("  Top courses:", ", ".join(top_courses))

In [21]:
find_topics(10,10,10)


🔹 Topic #1
  Top terms: 136, 118, 18h00, 172, 083, 164, mage, reserach, agt, 77
  Top courses: UE U : Cartography, Studio BA4 (De Vylder & Taillieu), Théorie et critique du projet MA1 (Gugger), Théorie et critique du projet MA2 (Geers), Théorie et critique du projet MA1 (Geers), Principles and Practicals in X-Ray Scattering, Limestone-Calcined Clay - Cement : Characterisation methods, Gödel and recursivity, Satellite communications  systems and networks, Studio MA2 (Escher et GuneWardena)

🔹 Topic #2
  Top terms: rotat, train, month, doctor, propos, candidaci, project, report, student, laboratori
  Top courses: Training Rotation (EDNE), Training Rotation (EDMS), Industrial electronics II, Practical - Cole Lab, Optical waves propagation, How people learn II, Biophysics II, Statistical thermodynamics, Integrated circuits technology, Nano CMOS Devices & Technologies for Tera-Bit Circuits and Systems

🔹 Topic #3
  Top terms: project, student, method, model, system, ic, the, learn, laborat

We Can label these 10 dominant concepts/ topics in the following way:
- Topic #1 => Thesis Projects 
- Topic #2 => Physics/ Thermodynamics
- Topic #3 => Semester Projects 
- Topic #4 => Modelling
- Topic #5 => Advanced Optics and Spectromatry 
- Topic #6 => Chemistry and Pharmacology 
- Topic #7 => Biology and Genetics
- Topic #8 => Energy
- Topic #9 => Optics and Lasers 
- Topic #10 => Electrical Engeneering

Analysis: The resulted groups are somewhat distinctive although we can see that they are infinging on each other a bit. I think that this may be an inherent property of LSI which don't specifically learn anything about the terms and documents themselves and purely relies on the SVD propreties. 

## 4.6 Document similarity search in concept space

In [22]:
#function to calculate cosine similarity between a term and a document 
#params: term: word representing a term (string) 
#        doc: name of a document (string)
# returns the similarity score of the given pair. 
def sim_on_term(term, doc):

    u_term = U[vocab_index[term], :]  
    v_doc = Vt[:, name_index[doc]]    
    
    norm_term = np.linalg.norm(u_term)
    norm_doc = np.linalg.norm(S @ v_doc)

    if norm_term == 0 or norm_doc == 0: return 0.0  

    num = u_term @ S @ v_doc
    return num / (norm_term * norm_doc)
    


In [23]:
# calclates the docs with the highest similarity score as a given object (term or course name) and prints the top 5 candidates
# param: obj: term or course name in string format
#        func: function to be used to calculate similarity (depends if obj in a terme or a course name)
def find_top5(obj,func):
    cos_sim = []
    for name in course_names:
         if name == obj: continue #skips identical queries (in doc to doc search)   
         cos_sim.append((name, func(obj,name)))
    sorted_sim = sorted(cos_sim, key=lambda x: x[1], reverse=True)[:5]
    print("Top 5 picks for",obj,"are:")
    for i in sorted_sim:
        print(i)
    print("\n")

In [24]:
find_top5("facebook",sim_on_term)
find_top5("markov",sim_on_term)
find_top5("chain",sim_on_term)

Top 5 picks for facebook are:
('Computational Social Media', 0.9233690816098991)
('Social media', 0.605095933131071)
('How people learn I', 0.4609564152810865)
('Internet analytics', 0.33860641253672763)
('How people learn II', 0.3272361897318959)


Top 5 picks for markov are:
('Applied stochastic processes', 0.704051730318847)
('Statistical Sequence Processing', 0.6986519338554791)
('Applied probability & stochastic processes', 0.6696221156252392)
('Markov chains and algorithmic applications', 0.49327318933699016)
('Fundamentals in statistical pattern recognition', 0.2789491251742943)


Top 5 picks for chain are:
('Supply chain management', 0.8232309923660509)
('Mathematical models in supply chain management', 0.7213647148259263)
('Operations: economics & strategy', 0.5662845045186415)
('Applied stochastic processes', 0.5321343134932818)
('Applied probability & stochastic processes', 0.4680315320741975)




The results are very satisfactory. 
- For "facebook" we see a list of courses with a great emphasis on people, media and communication which are all related to facebook. 
- As for "markov", being a russian mathematician who mostly contributed to the probabilistic/stochastic field, we see a list of courses that all are encompassing these concepts in one way or another. 

I belive that this function has brought up better results for the "facebook" query and similar results for the "markov" query. 

## 4.7 Document-document Similarity

Given two documets **$d$** and **$d'$**, the cosine similarity is defined as:

$$
\text{sim}(d, d') = \frac{\mathbf{v_{d}} S \mathbf{v_{d'}^\top}}{\|\mathbf{v_{d}}\| \cdot \|S\mathbf{{v_{d'}}}\|}
$$

Where:
- **$v_d$** and **$v_{d'}$** are the row vectors of the matrix **V** corresponding to the documents $d$ and $d'$ respectively.


In [25]:
#computes the similarity score between two course names (in string) 
#params: d1: course name 1
#        d2: course name 2
#returns: similarity score
def sim_on_doc(d1,d2):
    v1 = Vt[:,name_index[d1]]
    v2 = Vt[:,name_index[d2]]
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(S @v2)
    
    if norm1 == 0 or norm2 == 0: return 0.0
    
    num = v1 @ S @ v2
    return num / (norm1* norm2)
    

In [26]:
find_top5("Internet analytics",sim_on_doc)

Top 5 picks for Internet analytics are:
('Distributed information systems', 0.5023913425770477)
('Computational Social Media', 0.45363016779227094)
('A Network Tour of Data Science', 0.43093999637197666)
('Applied data analysis', 0.42888051671984206)
('Networks out of control', 0.3274893817901586)




The top 5 course picks are quite satisfactory. We see that all of the chosen courses are focused around networks, infortion systems, data analysis which are the core concepts of the "Internet analytics" class. 